In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('../data/intermediate/intermediate_tweets.csv', encoding='utf-8')

/Users/Isaacbolo/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13099356 entries, 0 to 13099355
Data columns (total 25 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   0       object
 1   1       object
 2   2       object
 3   3       object
 4   4       object
 5   5       object
 6   6       object
 7   7       object
 8   8       object
 9   9       object
 10  10      object
 11  11      object
 12  12      object
 13  13      object
 14  14      object
 15  15      object
 16  16      object
 17  17      object
 18  18      object
 19  19      object
 20  20      object
 21  21      object
 22  22      object
 23  23      object
 24  24      object
dtypes: object(25)
memory usage: 2.4+ GB


In [4]:
df.isnull().sum()

0      1941504
1      3477331
2      5124825
3      8024700
4     10503336
5     11558587
6     12194230
7     12547056
8     12808504
9     12965885
10    13031872
11    13068816
12    13084361
13    13092569
14    13096555
15    13098094
16    13098635
17    13099173
18    13099301
19    13099333
20    13099345
21    13099351
22    13099352
23    13099354
24    13099355
dtype: int64

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,Niall actually called Zayn when his daughter w...,Music Genre,Person,Music Album,Musician,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,They really be updating the Johnny Depp and Am...,Brand,Brand Category,Brand Vertical,Person,Actor,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The #USPS is also corrupt. They have gone pos...,Brand,Brand Category,Political Body,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"No matter what happens, I did get here and I d...",Actor,Person,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,@SoftlySteph This is a good tweet,Brand,Brand Category,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert to multiclass problem from a multilabel problem

In [6]:
# Drop rows will null tweets
df_notnull = df.dropna(subset=['0', '1'])
df_notnull.shape

(9622025, 25)

In [7]:
df.shape[0] - df_notnull.shape[0]

3477331

In [8]:
# Rename columns
rename = {'0': 'tweet', '1': 'context'}
df_notnull = df_notnull.rename(columns=rename)
df_notnull.columns

Index(['tweet', 'context', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24'],
      dtype='object')

In [9]:
df_ = df_notnull[['tweet', 'context']]
labels = df_['context'].unique()
labels

array(['Music Genre', 'Brand', 'Actor', 'Video Game',
       'Entities [Entity Service]', 'Holiday', 'Ongoing News Story',
       'Cities', 'Brand Category', 'Events [Entity Service]',
       'Brand Vertical', 'Musician', 'Politician',
       'Interests and Hobbies Vertical', 'Person', 'TV Shows',
       'Sports League', 'Interests and Hobbies Category',
       'Interests and Hobbies', 'States', 'Colleges & Universities',
       'Food', 'Multimedia Franchise', 'Reoccurring Trends',
       'Movie Genre', 'Sports Team', 'Sports Personality',
       'Baseball Game', 'Technology', 'Google Product Taxonomy',
       'Video Game Personality ', 'Book Genre', 'Fan Community',
       'Viral Accounts', 'Product', 'Movie', 'Exercise & fitness',
       'Athlete', 'Digital Creator', 'Political Body', 'Political Race',
       'Stocks', 'Award Show', 'Digital Assets & Crypto',
       'Global TV Show', 'Coach', 'Esports Team', 'Music Album',
       'TV Genres', 'Unified Twitter Taxonomy', 'Place', 'Spo

In [10]:
encoding = {label: i for i, label in enumerate(labels)}
encoding

{'Music Genre': 0,
 'Brand': 1,
 'Actor': 2,
 'Video Game': 3,
 'Entities [Entity Service]': 4,
 'Holiday': 5,
 'Ongoing News Story': 6,
 'Cities': 7,
 'Brand Category': 8,
 'Events [Entity Service]': 9,
 'Brand Vertical': 10,
 'Musician': 11,
 'Politician': 12,
 'Interests and Hobbies Vertical': 13,
 'Person': 14,
 'TV Shows': 15,
 'Sports League': 16,
 'Interests and Hobbies Category': 17,
 'Interests and Hobbies': 18,
 'States': 19,
 'Colleges & Universities': 20,
 'Food': 21,
 'Multimedia Franchise': 22,
 'Reoccurring Trends': 23,
 'Movie Genre': 24,
 'Sports Team': 25,
 'Sports Personality': 26,
 'Baseball Game': 27,
 'Technology': 28,
 'Google Product Taxonomy': 29,
 'Video Game Personality ': 30,
 'Book Genre': 31,
 'Fan Community': 32,
 'Viral Accounts': 33,
 'Product': 34,
 'Movie': 35,
 'Exercise & fitness': 36,
 'Athlete': 37,
 'Digital Creator': 38,
 'Political Body': 39,
 'Political Race': 40,
 'Stocks': 41,
 'Award Show': 42,
 'Digital Assets & Crypto': 43,
 'Global TV Sh

In [11]:
label_encodings = df_['context'].map(encoding)
label_encodings

0            0
1            1
2            1
3            2
4            1
            ..
13099351     0
13099352     0
13099353     0
13099354    26
13099355    22
Name: context, Length: 9622025, dtype: int64

In [12]:
df_['label'] = label_encodings
df_.head()

/Users/Isaacbolo/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet,context,label
0,Niall actually called Zayn when his daughter w...,Music Genre,0
1,They really be updating the Johnny Depp and Am...,Brand,1
2,The #USPS is also corrupt. They have gone pos...,Brand,1
3,"No matter what happens, I did get here and I d...",Actor,2
4,@SoftlySteph This is a good tweet,Brand,1


In [13]:
# Save encodings
import json
with open('../data/processed/context2label.json', 'w') as f:
    f.write(json.dumps(encoding))

In [31]:
!pip install tweet-preprocessor

In [14]:
import preprocessor as p
import re
p.set_options(p.OPT.URL, p.OPT.MENTION)

In [15]:
def tweet_preprocessor(text):
    return re.sub(r'#', '', p.clean(text)).lower()

In [16]:
clean_tweets = []
for tweet in tqdm(df_['tweet'].values):
    clean_tweets.append(tweet_preprocessor(tweet))
clean_tweets[:30]

  0%|          | 0/9622025 [00:00<?, ?it/s]

['niall actually called zayn when his daughter was born, streamed dusk till dawn on his live, made an effort to see him at the bmas after they separated and zayn said niall was his fave last year. yeah, i chose the right people to stan.',
 'they really be updating the johnny depp and amber heard case like an espn sports ticker, shit is insane.',
 'the usps is also corrupt. they have gone postal. we ought to close most of the list offices. let people use fedex or ups, or ponyexpress. never again.',
 'no matter what happens, i did get here and i did tell the truth. -johnny depp neverfeartruth truthwins mentoo verdictwatch',
 'this is a good tweet',
 'mommy yelan 原神 genshinimpact yelan',
 'i love bad comedy films so much like senior year is the best movie in cinematic history i am serious',
 "literally spent five minutes browsing the pridemonth. already insufferable corporate pandering, clickbait, and my favorite, just the most annoying people you hope to never meet. don't hate the real l

In [17]:
df_['tweet_clean'] = clean_tweets

/Users/Isaacbolo/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_.head()

,tweet,context,label,tweet_clean
0,Niall actually called Zayn when his daughter w...,Music Genre,0,niall actually called zayn when his daughter w...
1,They really be updating the Johnny Depp and Am...,Brand,1,they really be updating the johnny depp and am...
2,The #USPS is also corrupt. They have gone pos...,Brand,1,the usps is also corrupt. they have gone posta...
3,"No matter what happens, I did get here and I d...",Actor,2,"no matter what happens, i did get here and i d..."
4,@SoftlySteph This is a good tweet,Brand,1,this is a good tweet


In [19]:
df_.isnull().sum()

tweet          0
context        0
label          0
tweet_clean    0
dtype: int64

In [20]:
# Split data. Do one for a sample and complete dataset
from sklearn.model_selection import train_test_split

In [33]:
df_sample = df_.sample(1_000_000)
train_sm, test_sm = train_test_split(df_sample, test_size=1/3, shuffle=True)
train_lg, test_lg = train_test_split(df_, test_size=1/3, shuffle=True)

In [34]:
train_sm.reset_index(drop=True, inplace=True)
test_sm.reset_index(drop=True, inplace=True)
train_lg.reset_index(drop=True, inplace=True)
test_lg.reset_index(drop=True, inplace=True)

In [35]:
train_sm.shape[0], test_sm.shape[0], train_lg.shape[0], test_lg.shape[0]

(666666, 333334, 6414683, 3207342)

In [36]:
# check if there are any null values in any of the splits.
train_sm.isnull().any(), test_sm.isnull().any(), train_lg.isnull().any(), test_lg.isnull().any()

(tweet          False
 context        False
 label          False
 tweet_clean    False
 dtype: bool,
 tweet          False
 context        False
 label          False
 tweet_clean    False
 dtype: bool,
 tweet          False
 context        False
 label          False
 tweet_clean    False
 dtype: bool,
 tweet          False
 context        False
 label          False
 tweet_clean    False
 dtype: bool)

In [37]:
train_sm.head(3)

,tweet,context,label,tweet_clean
0,just finished downloading fall guys!!! gonna p...,Video Game,3,just finished downloading fall guys!!! gonna p...
1,Good thread on kink &amp; pride https://t.co/m...,Holiday,5,good thread on kink &amp; pride
2,$200 | 2.8 JT • 24 hrs • rt\n\n➡️ $100 - follo...,Technology,28,$200 | 2.8 jt • 24 hrs • rt ➡️ $100 - follow ✳...


Save as parquet

In [38]:
train_sm.to_parquet('../data/processed/sm/train_sm.gzip', index=None)
test_sm.to_parquet('../data/processed/sm/test_sm.gzip', index=None)

In [39]:
train_lg.to_parquet('../data/processed/lg/train_lg.gzip', index=None)
test_lg.to_parquet('../data/processed/lg/test_lg.gzip', index=None)

Save as csv, but this loses data

In [26]:
train_sm.to_csv('../data/processed/sm/train_sm.csv', index=None, encoding='utf-8')
test_sm.to_csv('../data/processed/sm/test_sm.csv', index=None, encoding='utf-8')

In [27]:
train_lg.to_csv('../data/processed/lg/train_lg.csv', index=None, encoding='utf-8')
test_lg.to_csv('../data/processed/lg/test_lg.csv', index=None, encoding='utf-8')